In [1]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor
import requests
import xml.etree.ElementTree as ET
import time

In [2]:
def con_cur_to_bgg_db():
    con = pg2.connect(host='34.216.22.202',
                  dbname='postgres',
                  user='postgres')
    cur = con.cursor(cursor_factory=RealDictCursor)
    return con, cur

def get_sql_unames(offset, limit):
    con, cur = con_cur_to_bgg_db()
    cur.execute('SELECT * FROM users OFFSET {} LIMIT {};'.format(offset, limit))
    ulist = cur.fetchall()
    con.commit()
    con.close()
    return ulist

def select_user(ulist, index):
    uid = ulist[index]['uid']
    uname = ulist[index]['uname']
    return uid, uname

def request_user_api(uname):
    url = 'http://www.boardgamegeek.com/xmlapi/collection/{}?rated=1'.format(uname)
    r = requests.get(url)
    if (r.status_code != 202) and (r.status_code != 200):
        print('{} error on {}'.format(r.status_code, uname))
    return r

def request_user_ratings(offset, limit):
    filename = 'ratings_log/{}_{}'.format(offset, limit)
    with open(filename, 'w') as f:
        f.write('{}\n'.format(time.ctime()))
    ulist = get_sql_unames(offset, limit)
    for index in range(len(ulist)):
        uid, uname = select_user(ulist, index)
        r = request_user_api(uname)
        start = time.time()
        with open(filename, 'a') as f:
            f.write('{}, {}\n'.format(r.status_code, uname))
        pause = 5 - (time.time() - start)
        if pause > 0:
            time.sleep(pause)
    with open(filename, 'a') as f:
        f.write(time.ctime())

In [71]:
request_user_ratings(offset=200, limit=100)

In [39]:
def get_user_api(uname):
    url = 'http://www.boardgamegeek.com/xmlapi/collection/{}?rated=1'.format(uname)
    r = requests.get(url)
    if (r.status_code != 202) and (r.status_code != 200):
        print('{} error on {}'.format(r.status_code, uname))
    while r.status_code == 202:
        time.sleep(5)
        r = requests.get(url)
    if (r.status_code != 202) and (r.status_code != 200):
        print('{} error on {}'.format(r.status_code, uname))
    elif r.status_code == 200:
        return r

def select_user(ulist, index):
    uid = ulist[index]['uid']
    uname = ulist[index]['uname']
    return uid, uname

def get_user_ratings(uid, r):
    tree = ET.fromstring(r.text)
    item_list = tree.findall('item')
    user_ratings = []
    for item in item_list:
        gid = item.attrib['objectid']
        rating = item.find('stats/rating').attrib['value']
        user_ratings.append((uid, gid, rating))
    return user_ratings

def insert_user_ratings(user_ratings):
    con, cur = con_cur_to_bgg_db()
    cur.execute('INSERT INTO ratings VALUES\
            {};'.format(", ".join(repr(e) for e in user_ratings)))
    con.commit()
    con.close()
    
def iterate_users(ulist):
    for index in range(len(ulist)):
        try:
            uid, uname = select_user(ulist, index)
            r = get_user_api(uname)
            start = time.time()
            user_ratings = get_user_ratings(uid, r)
            insert_user_ratings(user_ratings)
            pause = 5 - (time.time() - start)
            if pause > 0:
                time.sleep(pause)
        except:
            with open('ratings_log/get_errors', 'a') as f:
                f.write('Error on {}\n'.format(ulist[index]))

In [27]:
ulist = get_sql_unames(0, 4000)

In [ ]:
iterate_users(ulist)